In [1]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt


In [2]:

df = pd.read_feather('data/nq17-23_1min.feather')
df.index = df.index - pd.Timedelta(minutes=1)



In [68]:
tick_size = 0.25

df['body'] = ((df['close'] - df['open']) / tick_size).astype(int)
df['top_wick'] = ((df['high'] - df[['open', 'close']].max(axis=1)) / tick_size).astype(int)
df['bottom_wick'] = ((df[['open', 'close']].min(axis=1) - df['low']) / tick_size).astype(int)


In [70]:
# from statsmodels.tsa.stattools import adfuller

# result = adfuller(df['top_wick'].iloc[0:100000])
# print('ADF Statistic: %f' % result[0])
# print('p-value: %f' % result[1])
x
# # Вывод результатов
# for key, value in result[4].items():
#     print('\t%s: %.3f' % (key, value))

ADF Statistic: -17.580018
p-value: 0.000000
	1%: -3.430
	5%: -2.862
	10%: -2.567


In [8]:

# body_list = sorted(df[['body'].unique().tolist())

# #convert value to index
# body_to_index = {v: (i) for i, v in enumerate(body_list)}

# #convert index to value
# index_to_body = {i: v for i, v in enumerate(body_list)}

# #apply mapping of value to index
# df['candle_index'] = df['body'].map(body_to_index)



In [15]:
from sklearn.cluster import KMeans

# Предполагаем, что df - это ваш DataFrame
X = df[['volume', 'top_wick', 'body', 'bottom_wick']].to_numpy()

#standardize features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Количество квантилей
num_quantiles = 1024

# Применение K-means
cluster = KMeans(n_clusters=num_quantiles, random_state=0, n_init=10).fit_predict(X)


In [141]:

# Получение меток кластера для каждой свечи
df['cluster'] = cluster


In [142]:
print(df['cluster'].value_counts())
df[df['cluster'] == 12].head(10)

cluster
12     74247
478    51549
273    47018
699    44757
554    37758
       ...  
594        1
74         1
573        1
782        1
495        1
Name: count, Length: 1024, dtype: int64


,open,high,low,close,volume,body,top_wick,bottom_wick,cluster
date,,,,,,,,,
2017-01-02 18:20:00-05:00,4891.25,4891.50,4891.25,4891.50,23,1,0,0,12
2017-01-02 18:22:00-05:00,4890.75,4891.00,4890.75,4891.00,18,1,0,0,12
2017-01-02 18:25:00-05:00,4890.75,4891.00,4890.75,4891.00,19,1,0,0,12
2017-01-02 18:36:00-05:00,4890.50,4890.50,4890.50,4890.50,3,0,0,0,12
2017-01-02 18:37:00-05:00,4890.50,4890.50,4890.50,4890.50,6,0,0,0,12
2017-01-02 18:39:00-05:00,4890.75,4890.75,4890.75,4890.75,5,0,0,0,12
2017-01-02 18:40:00-05:00,4890.75,4891.00,4890.75,4891.00,12,1,0,0,12
2017-01-02 18:41:00-05:00,4891.00,4891.25,4891.00,4891.25,12,1,0,0,12
2017-01-02 18:47:00-05:00,4890.00,4890.00,4890.00,4890.00,32,0,0,0,12


In [2]:
#df.to_feather('data/nq17-23_1min_quantile_256.feather')
#df = pd.read_feather('data/nq17-23_1min_quantile.feather')

In [143]:
#merge clusters with low quantiles into one

low_quantiles = df['cluster'].value_counts()[df['cluster'].value_counts() <1000].index.to_list()
#df.loc[df[df['cluster'].isin(low_quantiles)].index,'cluster'] = df['cluster'].max() + 1

#re-cluster candles with low quantiles
X = df[df['cluster'].isin(low_quantiles)][['volume', 'top_wick', 'body', 'bottom_wick']].to_numpy()
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Количество квантилей
num_quantiles = 50

# Применение K-means
new_cluster = KMeans(n_clusters=num_quantiles, random_state=0, n_init=10).fit_predict(X)



In [144]:
np.unique(new_cluster, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
       dtype=int32),
 array([8180, 6672, 3679, 8099, 7446, 1915, 7281, 4379,  290, 6870, 1319,
        7495,  136,   16, 1493, 4931, 5876, 1173, 2025, 2956, 4738, 3340,
         658, 2834, 1308, 4561, 6863,  321, 6128, 5644, 2047, 7943,  348,
        7666,   18, 1461, 1359,  535, 6416, 7273, 2395,  734, 2066, 6741,
          75, 3313, 7447, 8530, 1805, 1646]))

In [145]:
df.loc[df[df['cluster'].isin(low_quantiles)].index,'cluster'] = df['cluster'].max() + 1 + new_cluster

In [146]:
# reindex clusters numbers to limit dict size
clusters_dict = list(df['cluster'].unique())
cluster_reindex = {v: (i) for i, v in enumerate(clusters_dict)}
df['cluster'] = df['cluster'].map(cluster_reindex)


In [149]:
print(df['cluster'].value_counts())
df[df['cluster'] == 49]

cluster
15     74247
8      51549
17     47018
11     44757
3      37758
       ...  
459      290
458      136
457       75
462       18
461       16
Name: count, Length: 463, dtype: int64


,open,high,low,close,volume,body,top_wick,bottom_wick,cluster
date,,,,,,,,,
2017-01-03 09:30:00-05:00,4898.00,4900.00,4892.00,4894.00,3071,-16,8,8,49
2017-01-05 15:59:00-05:00,4962.75,4964.00,4959.75,4960.50,2879,-9,5,3,49
2017-01-06 15:59:00-05:00,5004.50,5004.75,5002.00,5003.75,3020,-3,1,7,49
2017-01-09 10:45:00-05:00,5015.00,5015.75,5013.00,5013.75,2345,-5,3,3,49
2017-01-09 15:59:00-05:00,5022.50,5023.00,5020.00,5021.50,2984,-4,2,6,49
...,...,...,...,...,...,...,...,...,...
2023-10-26 09:36:00-04:00,14438.75,14441.75,14427.00,14429.00,2719,-39,12,8,49
2023-10-26 10:09:00-04:00,14311.50,14315.00,14304.00,14305.25,2492,-25,14,5,49
2023-10-26 11:05:00-04:00,14313.75,14315.75,14302.75,14304.25,2387,-38,8,6,49


In [150]:
sizes_mean = df.groupby('cluster').mean()[['top_wick', 'body', 'bottom_wick']]
sizes_median = df.groupby('cluster').median()[['top_wick', 'body', 'bottom_wick']]

In [151]:
#convert sizes median dataframe to dict with quantile as key abd tuple of sizes as value
index_to_candles = sizes_median.to_dict('index')
index_to_candles = {k: tuple(v.values()) for k, v in index_to_candles.items()}


In [152]:
def full_candle_restore(candle_index:(np.array,torch.Tensor), start_price:float, number_of_candles = None, index_map = index_to_candles) -> pd.DataFrame:
    '''
    Restore full candle from index of candle and start price
    '''
    if isinstance(candle_index, torch.Tensor):
        candle_index = candle_index.numpy()
    if number_of_candles is not None:
        candle_index = candle_index[:number_of_candles]
    
    candles = []
    for idx, cdl_idx in enumerate(candle_index):
        candle = {}
        if idx == 0:
            candle['open'] = start_price
        else:
            candle['open'] = candles[-1]['close']
        top_wick, body, bottom_wick = index_map[cdl_idx]
        close = candle['open'] + body * tick_size
        high = close + top_wick * tick_size if close > candle['open'] else candle['open'] + top_wick * tick_size
        low = candle['open'] - bottom_wick * tick_size if close > candle['open'] else close - bottom_wick * tick_size
        candle['high'] = high
        candle['low'] = low
        candle['close'] = close
        candles.append(candle)
    
    return pd.DataFrame(candles)

restored_df = full_candle_restore(df['cluster'].values, df['open'].values[0],number_of_candles=1000)

In [153]:
#plot original and restored candles using plotly
import plotly.graph_objects as go
import plotly.express as px

original_candles = df[['open', 'high','low','close']].iloc[:restored_df.shape[0]]

fig = go.Figure(data=[go.Candlestick(x=original_candles.index,
                open=original_candles['open'],
                high=original_candles['high'],
                low=original_candles['low'],
                close=original_candles['close'])])


fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

fig = go.Figure(data=[go.Candlestick(x=restored_df.index,
                open=restored_df['open'],
                high=restored_df['high'],
                low=restored_df['low'],
                close=restored_df['close'])])
fig.update_layout(xaxis_rangeslider_visible=False)
# add text lable witn cluster number above each candle
fig.add_trace(go.Scatter(x=restored_df.index, y=restored_df['high'] + 0.5, text=df['cluster'].values[:restored_df.shape[0]], mode="text"))

fig.show()

#plot original candles with cluster number 49
candles_49  = df[df['cluster'] == 49][['open', 'high','low','close']].iloc[:20]
fig = go.Figure(data=[go.Candlestick(x=candles_49.index,
                open=candles_49['open'],
                high=candles_49['high'],
                low=candles_49['low'],
                close=candles_49['close'])])
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()



In [154]:
#contstruct the tensor with  sequence of candles.
seq_columns = ['cluster']

seq_tensor = torch.tensor(df[seq_columns].values).reshape(-1)


In [155]:
seq_tensor = seq_tensor[~torch.isnan(seq_tensor)].to(torch.int64)


In [156]:

# save sequense tensor to hdf5
import h5py
with h5py.File('data/nq17-23_1min_candle_seq_463.hdf5', 'w') as f:
       dataset = f.create_dataset('data', shape=seq_tensor.shape, dtype='i8')
       dataset[:] = seq_tensor[:]
       #save index_to_candle dictionary
       idx_dataset = f.create_dataset('index_to_candle', shape=(len(index_to_candles),3), dtype='i8')
       for idx, candle in index_to_candles.items():
              idx_dataset[idx] = candle

       






In [59]:
struct_df = pd.read_feather('data/nq_struct_df.feather')

In [60]:
struct_df['move'] = struct_df['structure'] - struct_df['structure'].shift(1)
struct_df.fillna(0, inplace=True)

In [61]:
df = df.join(struct_df[['move','structure']])

In [62]:
df.bfill(inplace=True)

In [63]:
swing_len = 25
df.loc[df[(df['move'] >=swing_len) & (df['move'] != df['move'].shift(1))].index, 'trade'] = 1
df.loc[df[(df['move'] >=swing_len) & (df['move'] == df['move'].shift(1))].index, 'trade'] = 2
df.loc[df[(df['move'] >=swing_len) & (df['move'] != df['move'].shift(-1))].index, 'trade'] = 3

df.loc[df[(df['move'] <=-swing_len) & (df['move'] != df['move'].shift(1))].index, 'trade'] = 4
df.loc[df[(df['move'] <=-swing_len) & (df['move'] == df['move'].shift(1))].index, 'trade'] = 5
df.loc[df[(df['move'] <=-swing_len) & (df['move'] != df['move'].shift(-1))].index, 'trade'] = 6

df['trade'].fillna(0, inplace=True)


In [1]:
df

NameError: name 'df' is not defined